In [1]:
import pandas as pd

In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
#from sklearn_pandas import DataFrameMapper
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings('ignore')

In [3]:
import gensim.downloader as api

class MeanEmbeddingVectorizer(object):
    def __init__(self, word_model_vectors):
        self.word_model_vector = word_model_vectors
        self.vector_size = word_model_vectors.vector_size

    def fit(self): 
        return self

    def transform(self, docs):  
        doc_word_vector = np.vstack([self.word_average(sent) for sent in docs])
        return doc_word_vector

    def word_average(self, sent):
        mean = []
        for word in sent:
            if word in self.word_model_vector.index_to_key:
                mean.append(self.word_model_vector.get_vector(word))

        if not mean:  # empty words
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean

glove_vectors = api.load("glove-wiki-gigaword-50")
mev_glove = MeanEmbeddingVectorizer(glove_vectors)

In [11]:
df = pd.read_csv('/home/jovyan/__RHI/FINAL_PREPROCESSED_SKILLS_CLEANED.csv')

In [5]:
labeled_df = df.loc[df.hasSalary==True].reset_index(drop=True)

In [6]:
modelling_df = labeled_df.drop(columns=['url', 'title', 'location', 'salaryRange',
                                'maxSalary', 'currency', 'salaryPeriod', 'hasSalary',
                                          'company', 'position', 'datePosted', 'validThrough', 'jobDescription', 
                                           'skills', 'lang', 'titleLang', 'details'])

In [7]:
def filling_na(level):
    if pd.isna(level):
        return 'Unknown'
    else:
        return level

In [8]:
modelling_df['level'] = modelling_df['level'].apply(filling_na)

In [12]:
modelling_df.to_csv('/home/jovyan/__RHI/MODELING_DATASET_W_SALARY.csv', index=False)

In [14]:
modelling_df = pd.read_csv('/home/jovyan/__RHI/MODELING_DATASET_W_SALARY.csv')

In [16]:
modelling_df

,type,level,minSalary,.NET,3D Reconstruction,A/B testing,AGI,AI governance,AI strategy,AIStats,...,Unstructured data,VR,Visual SLAM,Weka,XGBoost,XML,ggplot2,spaCy,finalJobDescription,scope
0,Internship,Entry,9000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,creator open source python library looking mul...,data science
1,Full Time,Senior,55000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,hfr wa founded specializes area hedge fundalte...,data science
2,Full Time,Senior,150000.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,usa remote position position summary data lead...,management
3,Full Time,Mid,84000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,position social science researcher senior mana...,management
4,Full Time,Mid,81600.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,building new group ambitious independentminded...,research science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,Full Time,Senior,45000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,purpose maximize positive impact world capital...,ML engineering
2002,Full Time,Senior,50000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,centered data client need make confident decis...,data analytics
2003,Full Time,Unknown,50000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,global leader pioneer mobile webic format webt...,data analytics
2004,Full Time,Senior,76000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,ciampampt digital specialist partner endtoend ...,data engineering


In [9]:
X = modelling_df.drop('minSalary', axis = 1)
y = modelling_df.minSalary

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [59]:
X_train_vectorized = mev_glove.transform(X_train.finalJobDescription.str.split(' '))

X_test_vectorized = mev_glove.transform(X_test.finalJobDescription.str.split(' '))

In [12]:
from sklearn.model_selection import GridSearchCV, ParameterGrid

In [13]:
from catboost import CatBoostRegressor

In [57]:
params = dict(learning_rate=[.01, 0.05, .1, .15, .2, .25, .3, .5, 1],
              depth= [2, 4, 6, 8, 10],
              l2_leaf_reg= [0.2, 0.5, 1, 3])
                
grid = GridSearchCV(estimator=CatBoostRegressor(random_state=42, iterations=100, verbose=False),
                    param_grid=params,
                    scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_percentage_error'],
                    cv=5,
                    refit='neg_root_mean_squared_error',
                    verbose=5
)

In [60]:
grid.fit(X_train_vectorized, y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.432) neg_root_mean_squared_error: (test=-33931.596) total time=   0.1s
[CV 2/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.423) neg_root_mean_squared_error: (test=-30507.165) total time=   0.1s
[CV 3/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.408) neg_root_mean_squared_error: (test=-34681.822) total time=   0.1s
[CV 4/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.424) neg_root_mean_squared_error: (test=-33465.701) total time=   0.1s
[CV 5/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.445) neg_root_mean_squared_error: (test=-35646.603) total time=   0.1s
[CV 1/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.05; neg

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f0824be6940>,
             param_grid={'depth': [2, 4, 6, 8, 10],
                         'l2_leaf_reg': [0.2, 0.5, 1, 3],
                         'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25,
                                           0.3, 0.5, 1]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error',
                      'neg_mean_absolute_percentage_error'],
             verbose=5)

In [61]:
res = pd.DataFrame(grid.cv_results_)[['params', 'mean_test_neg_root_mean_squared_error',
                               'mean_test_neg_mean_absolute_percentage_error']].sort_values(
    'mean_test_neg_root_mean_squared_error', ascending=False)

res

,params,mean_test_neg_root_mean_squared_error,mean_test_neg_mean_absolute_percentage_error
119,"{'depth': 8, 'l2_leaf_reg': 0.5, 'learning_rat...",-27091.684955,-0.334189
138,"{'depth': 8, 'l2_leaf_reg': 3, 'learning_rate'...",-27375.716109,-0.340866
173,"{'depth': 10, 'l2_leaf_reg': 3, 'learning_rate...",-27405.019222,-0.344608
165,"{'depth': 10, 'l2_leaf_reg': 1, 'learning_rate...",-27417.440265,-0.339892
110,"{'depth': 8, 'l2_leaf_reg': 0.2, 'learning_rat...",-27444.466222,-0.340703
...,...,...,...
53,"{'depth': 4, 'l2_leaf_reg': 0.5, 'learning_rat...",-36827.250525,-0.467042
89,"{'depth': 6, 'l2_leaf_reg': 0.5, 'learning_rat...",-37306.686565,-0.476190
116,"{'depth': 8, 'l2_leaf_reg': 0.2, 'learning_rat...",-37381.774751,-0.467850
80,"{'depth': 6, 'l2_leaf_reg': 0.2, 'learning_rat...",-37961.590539,-0.478701


In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

In [62]:
rmse = mean_squared_error(y_test, grid.predict(X_test_vectorized), squared=False)
rmse

26688.335400387787

In [63]:
mape = mean_absolute_percentage_error(y_test, grid.predict(X_test_vectorized))
mape

0.3386366803829351

## Adding other non-text features

In [15]:
from sklearn.base import BaseEstimator, TransformerMixin

In [23]:
class Embedder(BaseEstimator, TransformerMixin):
    def __init__(self, col_name):
        self.col_name = col_name
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        Xt = X.copy()
        
        tr = mev_glove.transform(Xt[self.col_name].str.split(' '))
        
        Xt = pd.concat([Xt.reset_index(drop=True).drop(columns=self.col_name), pd.DataFrame(tr)], axis=1)
        
        #Xt.drop(columns=self.col_name, inplace=True)
        
        return Xt

In [89]:
embedder = Embedder('finalJobDescription')

embedder.fit_transform(X_train)

,0,1,2,3,4,5,6,7,8,9,...,340,341,342,343,344,345,346,347,348,349
0,Full Time,Senior,0,0,0,0,0,0,0,0,...,0.075748,-0.196206,0.103760,0.228870,-0.114911,-0.017496,0.210977,0.315589,0.062624,0.142242
1,Full Time,Senior,0,0,0,0,0,0,0,0,...,-0.035356,-0.200940,0.059467,0.146852,-0.084695,-0.139670,0.048423,0.411206,0.075738,0.168076
2,Full Time,Senior,0,0,0,0,0,0,0,0,...,0.109810,-0.222038,0.000283,0.263295,-0.066648,-0.093287,0.279083,0.469280,0.080943,0.258135
3,Full Time,Senior,0,0,0,0,0,0,0,0,...,0.098745,-0.203745,-0.024682,0.139685,0.008242,0.048132,0.186344,0.390628,-0.042655,0.171870
4,Full Time,Senior,0,0,1,0,0,0,0,0,...,0.026015,-0.208839,0.010041,0.272201,-0.027313,-0.050137,0.206530,0.370125,0.042266,0.219731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599,Full Time,Senior,0,0,0,0,0,0,0,0,...,0.117695,-0.194423,0.050657,0.285432,-0.161486,-0.034260,0.157636,0.355640,0.052148,0.217064
1600,Full Time,Senior,0,0,0,0,0,0,0,0,...,0.023175,-0.175616,-0.010676,0.210708,-0.099242,0.003720,0.102111,0.257122,-0.012542,0.273713
1601,Full Time,Senior,0,0,0,0,0,0,0,0,...,0.108541,-0.241845,0.060181,0.283232,-0.137510,-0.044869,0.252781,0.466786,0.031941,0.150529
1602,Full Time,Senior,0,0,0,0,0,0,0,0,...,0.016781,-0.148553,0.105073,0.270343,-0.130962,-0.104605,0.102985,0.320828,0.002801,0.169156


In [24]:
embedder = Embedder('finalJobDescription')

preprocessor = ColumnTransformer([("poly", PolynomialFeatures(degree=2, interaction_only=True), slice(2, 298)),
                          ("cat_encoder", OneHotEncoder(sparse=False, drop='first'), ['level', 'scope']),
                          ("type_encoder", OneHotEncoder(sparse=False, handle_unknown='ignore'), ['type']),        
                          ], 
                          remainder='passthrough')

prep_pipe = Pipeline([('embedder', embedder), 
                      ('preprocessor', preprocessor)
                              
                             ])

In [25]:
a = embedder.fit_transform(X_test)

In [27]:
a.iloc[:, 2:298]

,.NET,3D Reconstruction,A/B testing,AGI,AI governance,AI strategy,AIStats,ANN,API Development,APIs,...,UX,UX Research,Unstructured data,VR,Visual SLAM,Weka,XGBoost,XML,ggplot2,spaCy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
399,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [28]:
prep_pipe.fit(X_train)

Pipeline(steps=[('embedder', Embedder(col_name='finalJobDescription')),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('poly',
                                                  PolynomialFeatures(interaction_only=True),
                                                  slice(2, 298, None)),
                                                 ('cat_encoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['level', 'scope']),
                                                 ('type_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['type'])]))])

In [29]:
X_train_prep = prep_pipe.transform(X_train)
X_test_prep = prep_pipe.transform(X_test)

In [30]:
params = dict(learning_rate=[.25],
              depth= [4],
              l2_leaf_reg= [1])
                
grid = GridSearchCV(estimator=CatBoostRegressor(random_state=42, iterations=100, verbose=False),
                    param_grid=params,
                    scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_percentage_error'],
                    cv=5,
                    refit='neg_root_mean_squared_error',
                    verbose=5
)

In [31]:
grid.fit(X_train_prep, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END depth=4, l2_leaf_reg=1, learning_rate=0.25; neg_mean_absolute_percentage_error: (test=-0.241) neg_root_mean_squared_error: (test=-22738.506) total time=  26.6s
[CV 2/5] END depth=4, l2_leaf_reg=1, learning_rate=0.25; neg_mean_absolute_percentage_error: (test=-0.242) neg_root_mean_squared_error: (test=-23252.928) total time=  25.9s
[CV 3/5] END depth=4, l2_leaf_reg=1, learning_rate=0.25; neg_mean_absolute_percentage_error: (test=-0.258) neg_root_mean_squared_error: (test=-27587.014) total time=  26.5s
[CV 4/5] END depth=4, l2_leaf_reg=1, learning_rate=0.25; neg_mean_absolute_percentage_error: (test=-0.277) neg_root_mean_squared_error: (test=-26573.816) total time=  26.1s
[CV 5/5] END depth=4, l2_leaf_reg=1, learning_rate=0.25; neg_mean_absolute_percentage_error: (test=-0.271) neg_root_mean_squared_error: (test=-27624.702) total time=  27.2s


GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7fa3b70941c0>,
             param_grid={'depth': [4], 'l2_leaf_reg': [1],
                         'learning_rate': [0.25]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error',
                      'neg_mean_absolute_percentage_error'],
             verbose=5)

In [32]:
pd.DataFrame(grid.cv_results_)[['params', 'mean_test_neg_root_mean_squared_error', 
                                    'mean_test_neg_mean_absolute_percentage_error']].sort_values(
    'mean_test_neg_root_mean_squared_error', ascending=False).style.bar(vmin=0, vmax=1)

,params,mean_test_neg_root_mean_squared_error,mean_test_neg_mean_absolute_percentage_error
0,"{'depth': 4, 'l2_leaf_reg': 1, 'learning_rate': 0.25}",-25555.393215,-0.257644


In [33]:
rmse = mean_squared_error(y_test, grid.predict(X_test_prep), squared=False)
rmse

23656.46310066872

In [35]:
mape = mean_absolute_percentage_error(y_test, grid.predict(X_test_prep))
mape

0.2618170180693243

## GloVE + all other features extensive GridSearch

In [36]:
params = dict(learning_rate=[.01, 0.05, .1, .15, .2, .25, .3, .5, 1],
              depth= [2, 4, 6, 8, 10],
              l2_leaf_reg= [0.2, 0.5, 1, 3])
                
grid = GridSearchCV(estimator=CatBoostRegressor(random_state=42, iterations=100, verbose=False),
                    param_grid=params,
                    scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_percentage_error'],
                    cv=5,
                    refit='neg_root_mean_squared_error',
                    verbose=5
)

In [37]:
grid.fit(X_train_prep, y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.343) neg_root_mean_squared_error: (test=-30575.671) total time=  26.2s
[CV 2/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.320) neg_root_mean_squared_error: (test=-27936.329) total time=  26.2s
[CV 3/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.339) neg_root_mean_squared_error: (test=-32652.704) total time=  26.0s
[CV 4/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.329) neg_root_mean_squared_error: (test=-30667.790) total time=  25.8s
[CV 5/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.355) neg_root_mean_squared_error: (test=-32773.874) total time=  27.1s
[CV 1/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.05; neg

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7fa3b45fe760>,
             param_grid={'depth': [2, 4, 6, 8, 10],
                         'l2_leaf_reg': [0.2, 0.5, 1, 3],
                         'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25,
                                           0.3, 0.5, 1]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error',
                      'neg_mean_absolute_percentage_error'],
             verbose=5)

In [38]:
pd.DataFrame(grid.cv_results_)[['params', 'mean_test_neg_root_mean_squared_error', 
                                    'mean_test_neg_mean_absolute_percentage_error']].sort_values(
    'mean_test_neg_root_mean_squared_error', ascending=False).style.bar(vmin=0, vmax=1)

,params,mean_test_neg_root_mean_squared_error,mean_test_neg_mean_absolute_percentage_error
156,"{'depth': 10, 'l2_leaf_reg': 0.5, 'learning_rate': 0.15}",-23628.385444,-0.222377
167,"{'depth': 10, 'l2_leaf_reg': 1, 'learning_rate': 0.25}",-23683.179166,-0.221691
159,"{'depth': 10, 'l2_leaf_reg': 0.5, 'learning_rate': 0.3}",-23694.323977,-0.227309
158,"{'depth': 10, 'l2_leaf_reg': 0.5, 'learning_rate': 0.25}",-23718.819961,-0.229295
165,"{'depth': 10, 'l2_leaf_reg': 1, 'learning_rate': 0.15}",-23936.839756,-0.226241
146,"{'depth': 10, 'l2_leaf_reg': 0.2, 'learning_rate': 0.1}",-23966.838374,-0.232645
176,"{'depth': 10, 'l2_leaf_reg': 3, 'learning_rate': 0.25}",-24035.959113,-0.234394
147,"{'depth': 10, 'l2_leaf_reg': 0.2, 'learning_rate': 0.15}",-24082.858150,-0.224784
113,"{'depth': 8, 'l2_leaf_reg': 0.2, 'learning_rate': 0.25}",-24086.984380,-0.229841
149,"{'depth': 10, 'l2_leaf_reg': 0.2, 'learning_rate': 0.25}",-24094.170393,-0.225934


In [40]:
rmse = mean_squared_error(y_test, grid.predict(X_test_prep), squared=False)
rmse

22809.716479477156

In [41]:
mape = mean_absolute_percentage_error(y_test, grid.predict(X_test_prep))
mape

0.23663338797485453

## TFIDF + catboost

In [45]:

preprocessor = ColumnTransformer([("poly", PolynomialFeatures(degree=2, interaction_only=True), slice(2, 298)),
                           ("tfidf", TfidfVectorizer(ngram_range=(1, 2), max_df=0.97), 'finalJobDescription'),
                          ("cat_encoder", OneHotEncoder(sparse=False, drop='first'), ['level', 'scope']),
                          ("type_encoder", OneHotEncoder(sparse=False, handle_unknown='ignore'), ['type']),        
                          ], 
                          remainder='passthrough')

params = dict(learning_rate=[.15],
              depth= [8],
              l2_leaf_reg= [1])
                
grid = GridSearchCV(estimator=CatBoostRegressor(random_state=42, iterations=100, verbose=False),
                    param_grid=params,
                    scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_percentage_error'],
                    cv=5,
                    refit='neg_root_mean_squared_error',
                    verbose=5
)


In [43]:
X_train_proc = preprocessor.fit_transform(X_train)

In [44]:
X_test_proc = preprocessor.transform(X_test)

In [46]:
grid.fit(X_train_proc, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END depth=8, l2_leaf_reg=1, learning_rate=0.15; neg_mean_absolute_percentage_error: (test=-0.171) neg_root_mean_squared_error: (test=-17142.426) total time= 3.4min
[CV 2/5] END depth=8, l2_leaf_reg=1, learning_rate=0.15; neg_mean_absolute_percentage_error: (test=-0.160) neg_root_mean_squared_error: (test=-16919.159) total time= 3.5min
[CV 3/5] END depth=8, l2_leaf_reg=1, learning_rate=0.15; neg_mean_absolute_percentage_error: (test=-0.181) neg_root_mean_squared_error: (test=-19507.620) total time= 3.6min
[CV 4/5] END depth=8, l2_leaf_reg=1, learning_rate=0.15; neg_mean_absolute_percentage_error: (test=-0.170) neg_root_mean_squared_error: (test=-18708.633) total time= 3.6min
[CV 5/5] END depth=8, l2_leaf_reg=1, learning_rate=0.15; neg_mean_absolute_percentage_error: (test=-0.189) neg_root_mean_squared_error: (test=-22288.072) total time= 3.6min


GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7fa3e2251940>,
             param_grid={'depth': [8], 'l2_leaf_reg': [1],
                         'learning_rate': [0.15]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error',
                      'neg_mean_absolute_percentage_error'],
             verbose=5)

In [47]:
rmse = mean_squared_error(y_test, grid.predict(X_test_proc), squared=False)
rmse

17278.158490024412

In [48]:
mape = mean_absolute_percentage_error(y_test, grid.predict(X_test_proc))
mape

0.17949071420073962

## Word2Vec

In [18]:
X_train

,type,level,.NET,3D Reconstruction,A/B testing,AGI,AI governance,AI strategy,AIStats,ANN,...,Unstructured data,VR,Visual SLAM,Weka,XGBoost,XML,ggplot2,spaCy,finalJobDescription,scope
704,Full Time,Senior,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,believe better way choose transparent le confu...,data engineering
1136,Full Time,Senior,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,techenabled logistics mission provide hasslefr...,data analytics
828,Full Time,Senior,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,global blockchain behind world largest digital...,data science
159,Full Time,Senior,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,bosch global software technology private limit...,data engineering
813,Full Time,Senior,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,building world talent e first talent lifecycle...,data science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,Full Time,Senior,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,owns operates different across country includi...,data science
1216,Full Time,Senior,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,bosch shape future highquality technological i...,data science
1653,Full Time,Senior,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,collaborate customer gather requirement unders...,data engineering
559,Full Time,Senior,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,technology make world work everyone people mak...,data engineering


In [26]:
X_train.loc[704]

type                                                           Full Time
level                                                             Senior
.NET                                                                   0
3D Reconstruction                                                      0
A/B testing                                                            0
                                             ...                        
XML                                                                    0
ggplot2                                                                0
spaCy                                                                  0
finalJobDescription    believe better way choose transparent le confu...
scope                                                   data engineering
Name: 704, Length: 300, dtype: object

In [27]:
X_train['fjd'] = X_train['finalJobDescription'].str.split(' ')


In [28]:
X_train['fjd']

704     [believe, better, way, choose, transparent, le...
1136    [techenabled, logistics, mission, provide, has...
828     [global, blockchain, behind, world, largest, d...
159     [bosch, global, software, technology, private,...
813     [building, world, talent, e, first, talent, li...
                              ...                        
835     [owns, operates, different, across, country, i...
1216    [bosch, shape, future, highquality, technologi...
1653    [collaborate, customer, gather, requirement, u...
559     [technology, make, world, work, everyone, peop...
684     [role, looking, data, engineer, ii, help, buil...
Name: fjd, Length: 1604, dtype: object

In [30]:
import gensim

w2v_model = gensim.models.Word2Vec(X_train['fjd'],
                                   vector_size=10,
                                   window=5, # context
                                   min_count=2,
                                   sg=0, # 0=CBOW, 1=Skip-gram
                                   epochs=5)

w2v_model.save('word2vec_model')

In [31]:
w2v_words = set(w2v_model.wv.index_to_key)

print(len(w2v_words))

w2v_model.wv.index_to_key[:30]

15221


['data',
 'team',
 'experience',
 'work',
 'product',
 'working',
 'learning',
 'technology',
 'year',
 'solution',
 'science',
 'skill',
 'people',
 'analytics',
 'new',
 'engineering',
 'customer',
 'u',
 'model',
 'development',
 'build',
 'role',
 'help',
 'tool',
 'machine',
 'support',
 'platform',
 'ampamp',
 'including',
 'ee']

In [32]:
w2v_model.wv.get_vector('skill')

array([-3.2070737 , -0.28578496,  6.137533  ,  0.8489439 ,  0.21832451,
       -2.1366882 , -1.6038852 ,  5.31864   , -0.48933843, -3.038796  ],
      dtype=float32)

In [34]:
word_vector = np.array([np.array([w2v_model.wv[i] for i in ls if i in w2v_words]) for ls in X_train['fjd']], 
                       dtype=object)

word_vector[:1]

array([array([[-1.0331645 , -7.0942907 , -1.2222389 , ...,  0.9841872 ,
               -3.590413  , -0.7450138 ],
              [ 1.2954022 , -4.9475226 , -0.77080035, ...,  1.2306948 ,
               -3.2568486 , -0.81573623],
              [ 0.6775806 , -4.4810615 , -0.10819216, ...,  1.7995946 ,
               -1.3468567 , -2.6991537 ],
              ...,
              [-0.8257962 , -2.1643054 , -0.34828246, ..., -0.37075508,
               -1.6720536 , -0.09531414],
              [-0.52667326, -4.4205666 , -1.2896235 , ..., -1.7090579 ,
               -3.9360118 , -0.67850393],
              [-0.8577013 , -1.5142866 ,  0.13163954, ..., -0.728454  ,
               -1.4005778 , -0.6221054 ]], dtype=float32)              ],
      dtype=object)

In [35]:
# Not each word has a vector

for i, v in enumerate(word_vector):
    print(len(X_train['fjd'].iloc[i]), len(v))

328 328
255 233
324 314
190 187
597 586
428 406
443 434
124 120
292 271
345 319
265 254
257 257
301 300
352 352
368 341
352 352
261 240
526 519
566 548
156 154
246 220
326 326
255 242
562 549
439 439
86 85
663 663
368 347
402 402
494 469
165 163
460 460
306 295
309 287
457 450
392 391
439 439
412 396
306 264
340 332
262 247
347 345
439 411
353 352
422 422
378 365
111 109
258 257
392 363
438 421
418 381
338 338
1050 989
144 137
190 182
398 391
528 523
299 299
401 368
382 376
356 353
460 458
340 340
109 107
380 376
609 594
424 424
341 341
255 235
392 390
469 449
297 275
226 221
498 498
568 566
261 254
219 200
186 167
398 389
118 113
307 301
422 422
379 352
196 174
407 406
208 191
177 177
519 514
281 281
246 246
349 330
469 468
413 407
111 94
664 657
274 259
660 626
423 423
359 346
309 293
629 616
301 300
483 461
342 305
303 300
611 589
417 415
290 288
488 476
173 167
328 328
176 158
560 556
546 541
377 375
213 213
508 488
461 444
515 461
208 202
274 267
267 267
340 336
625 625
578 567
46

In [36]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word_model_vectors):
        self.word_model_vector = word_model_vectors
        self.vector_size = word_model_vectors.vector_size

    def fit(self): 
        return self

    def transform(self, docs):  
        doc_word_vector = np.vstack([self.word_average(sent) for sent in docs])
        return doc_word_vector

    def word_average(self, sent):
        mean = []
        for word in sent:
            if word in self.word_model_vector.index_to_key:
                mean.append(self.word_model_vector.get_vector(word))

        if not mean:  # empty words
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean

In [37]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2v_model.wv)

X_train_w2v_vectorized = mean_embedding_vectorizer.transform(X_train['fjd'])

X_train_w2v_vectorized[:1]

array([[ 0.79149777,  0.00744591,  1.9789678 , -0.46381047,  0.3741124 ,
         0.21263367,  1.4501076 ,  0.9704187 , -1.6418366 , -0.9565927 ]],
      dtype=float32)

In [64]:
l = []
for x in X_train_w2v_vectorized:
    l.append(len(x))

In [39]:
X_test['fjd'] = X_test['finalJobDescription'].str.split(' ')


In [40]:
X_test_w2v_vectorized = mean_embedding_vectorizer.transform(X_test['fjd'])

X_test_w2v_vectorized[:1]

array([[ 0.602481  , -0.56324327,  2.0942934 , -0.5213157 ,  0.1309992 ,
         0.39351305,  1.1375864 ,  1.1134597 , -1.6016983 , -0.77536863]],
      dtype=float32)

In [41]:
from sklearn.model_selection import GridSearchCV, ParameterGrid

In [42]:
from catboost import CatBoostRegressor

In [51]:
params = dict(learning_rate=[.01, 0.05, .1, .15, .2, .25, .3, .5, 1],
              depth= [2, 4, 6, 8, 10],
              l2_leaf_reg= [0.2, 0.5, 1, 3])
                
grid = GridSearchCV(estimator=CatBoostRegressor(random_state=42, iterations=100, verbose=False),
                    param_grid=params,
                    scoring=['neg_root_mean_squared_error', 'neg_mean_absolute_percentage_error'],
                    cv=5,
                    refit='neg_root_mean_squared_error',
                    verbose=5
)

In [52]:
grid.fit(X_train_w2v_vectorized, y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.434) neg_root_mean_squared_error: (test=-33603.662) total time=   0.4s
[CV 2/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.419) neg_root_mean_squared_error: (test=-30530.335) total time=   0.2s
[CV 3/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.412) neg_root_mean_squared_error: (test=-34579.381) total time=   0.5s
[CV 4/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.428) neg_root_mean_squared_error: (test=-33734.731) total time=   0.1s
[CV 5/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.01; neg_mean_absolute_percentage_error: (test=-0.447) neg_root_mean_squared_error: (test=-35689.067) total time=   0.2s
[CV 1/5] END depth=2, l2_leaf_reg=0.2, learning_rate=0.05; neg

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f0824f6a820>,
             param_grid={'depth': [2, 4, 6, 8, 10],
                         'l2_leaf_reg': [0.2, 0.5, 1, 3],
                         'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25,
                                           0.3, 0.5, 1]},
             refit='neg_root_mean_squared_error',
             scoring=['neg_root_mean_squared_error',
                      'neg_mean_absolute_percentage_error'],
             verbose=5)

In [53]:
res = pd.DataFrame(grid.cv_results_)[['params', 'mean_test_neg_root_mean_squared_error',
                               'mean_test_neg_mean_absolute_percentage_error']].sort_values(
    'mean_test_neg_root_mean_squared_error', ascending=False)

res

,params,mean_test_neg_root_mean_squared_error,mean_test_neg_mean_absolute_percentage_error
174,"{'depth': 10, 'l2_leaf_reg': 3, 'learning_rate...",-28392.268757,-0.341493
164,"{'depth': 10, 'l2_leaf_reg': 1, 'learning_rate...",-28451.893266,-0.340485
173,"{'depth': 10, 'l2_leaf_reg': 3, 'learning_rate...",-28465.377275,-0.343909
110,"{'depth': 8, 'l2_leaf_reg': 0.2, 'learning_rat...",-28498.493154,-0.346488
145,"{'depth': 10, 'l2_leaf_reg': 0.2, 'learning_ra...",-28573.369086,-0.347584
...,...,...,...
89,"{'depth': 6, 'l2_leaf_reg': 0.5, 'learning_rat...",-36813.073977,-0.460199
152,"{'depth': 10, 'l2_leaf_reg': 0.2, 'learning_ra...",-37135.694277,-0.431367
116,"{'depth': 8, 'l2_leaf_reg': 0.2, 'learning_rat...",-37217.411426,-0.457573
80,"{'depth': 6, 'l2_leaf_reg': 0.2, 'learning_rat...",-37613.386106,-0.443959


In [54]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

In [55]:
rmse = mean_squared_error(y_test, grid.predict(X_test_w2v_vectorized), squared=False)
rmse

27633.032603003652

In [56]:
mape = mean_absolute_percentage_error(y_test, grid.predict(X_test_w2v_vectorized))
mape

0.34679788899514324